In [1]:
import requests
import lxml
import sys

# 爬虫
from bs4 import BeautifulSoup
import xlwt
import time
import urllib

import pandas as pd
import numpy

import re


In [9]:
# 把中文字弄成一排 list 

def get_chinese_list(lst):
    temp = []
    for item in lst:
        temp.append(item.text+"; ")
    return (list(set(temp)))


In [12]:
count = 1

def craw(url,key_word,x):
    global count 
    
    User_Agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:56.0) Gecko/20100101 Firefox/56.0'
    re = r'https://www.qcc.com/search?key='+(urllib.parse.quote(key_word))
    print ("NO.", count, "key_word", key_word)
    count+=1 
    print ("re", re)
    headers = {
            'Host':'www.qcc.com',
            'Connection': 'keep-alive',
            'Accept':r'text/html, */*; q=0.01',
            'X-Requested-With': 'XMLHttpRequest',
            'User-Agent':r'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
            'Referer': re,
            'Accept-Encoding':'gzip, deflate, br',
            'Accept-Language':'zh-CN,zh;q=0.9',
            'Cookie':r'QCCSESSID=c851f4tavl4mba6m2sjulr7a50; zg_did=%7B%22did%22%3A%20%22172f9271efd9bf-07ab9a761dcde9-31607402-1ea000-172f9271efe91e%22%7D; UM_distinctid=172f927206bd30-0ee7a6ec0eb8f4-31607402-1ea000-172f927206cc22; _uab_collina=159331799286569807742961; Hm_lvt_78f134d5a9ac3f92524914d0247e70cb=1593317994,1593324245,1593327257,1593331291; acw_tc=2ff6129915933988027872772e9a1d8029df1a9711fc52c4da655e12b2; CNZZDATA1254842228=1324649481-1593313536-https%253A%252F%252Fwww.google.com%252F%7C1593395960; hasShow=1; Hm_lpvt_78f134d5a9ac3f92524914d0247e70cb=1593398910; zg_de1d1a35bfa24ce29bbf2c7eb17e6c4f=%7B%22sid%22%3A%201593398807053%2C%22updated%22%3A%201593399015008%2C%22info%22%3A%201593317990148%2C%22superProperty%22%3A%20%22%7B%7D%22%2C%22platform%22%3A%20%22%7B%7D%22%2C%22utm%22%3A%20%22%7B%7D%22%2C%22referrerDomain%22%3A%20%22%22%2C%22cuid%22%3A%20%22e9bfdd942828258fd524e731d7aaddeb%22%2C%22zs%22%3A%200%2C%22sc%22%3A%200%7D',
            }
 
    try:
        response = requests.get(url,headers = headers)
        if response.status_code != 200:
            response.encoding = 'utf-8'
            print(response.status_code)
            print('ERROR')    
        soup = BeautifulSoup(response.text,'lxml')
    except Exception:
        print('请求都不让，这企查查是想逆天吗？？？')
    try:
        com_all_info = soup.find_all(class_='m_srchList')[0].tbody
        com_all_info_array = com_all_info.select('tr')
        
        print('开始爬取数据，请勿打开excel')
        
#         for i in range(0,len(com_all_info_array)):

        for i in range(0,1):

#            try:
                temp_g_name = com_all_info_array[i].select('td')[2].select('.ma_h1')[0].text    #获取公司名
                temp_g_tag = com_all_info_array[i].select('td')[2].select('.search-tags')[0].text    #获取公司标签
                temp_r_name = com_all_info_array[i].select('td')[2].select('p')[0].a.text    #获取法人名
                temp_g_money = com_all_info_array[i].select('td')[2].select('p')[0].select('span')[0].text.strip('注册资本：')    #获取注册资本
                temp_g_date = com_all_info_array[i].select('td')[2].select('p')[0].select('span')[1].text.strip('成立日期：')    #获取公司注册时间
                temp_r_email = com_all_info_array[i].select('td')[2].select('p')[1].text.split('\n')[1].strip().strip('邮箱：')    #获取法人Email
                temp_r_phone = com_all_info_array[i].select('td')[2].select('p')[1].select('.m-l')[0].text.strip('电话：')    #获取法人手机号
                temp_g_addr = com_all_info_array[i].select('td')[2].select('p')[2].text.strip().strip('地址：')    #获取公司地址
                temp_g_state = com_all_info_array[i].select('td')[3].select('.nstatus.text-success-lt.m-l-xs')[0].text.strip()  #获取公司状态
                
                website = com_all_info_array[i].find_all("a", class_='ma_h1')[0].get("href")
                website = "https://www.qcc.com/" + website
                
                print("$$$$$$$$$$$", website)
                                
                response = requests.get(website,headers = headers)
                if response.status_code != 200:
                    response.encoding = 'utf-8'
                    print(response.status_code)
                    print('ERROR')    
                soup = BeautifulSoup(response.text,'lxml')
        
                temp_g_gudong = get_chinese_list(soup.find_all("h3", class_='seo font-14'))
                
                print ("GUDONG: ", temp_g_gudong)
                
                g_name_list.append(temp_g_name)
                g_tag_list.append(temp_g_tag)
                r_name_list.append(temp_r_name)
                g_money_list.append(temp_g_money)
                g_date_list.append(temp_g_date)
                r_email_list.append(temp_r_email)
                r_phone_list.append(temp_r_phone)
                g_addr_list.append(temp_g_addr)
                g_state_list.append(temp_g_state)
                g_website_list.append(website) 
                g_gudong_list.append(temp_g_gudong)
                g_num_qcc.append(len(com_all_info_array))

    except Exception:
        no_result_list.append(key_word)
        print('好像被拒绝访问了呢...请稍后再试叭...')
         
if __name__ == '__main__':
    global g_name_list
    global g_tag_list
    global r_name_list
    global g_money_list
    global g_date_list
    global r_email_list
    global r_phone_list
    global g_addr_list
    global g_state_list
    global g_gudong_list
    global g_website_list
    global g_original_name
    global g_num_qcc
    global no_result_list
    
    g_name_list=[]
    g_tag_list=[]
    r_name_list=[]
    g_money_list=[]
    g_date_list=[]
    r_email_list=[]
    r_phone_list=[]
    g_addr_list=[]
    g_state_list=[]
    g_gudong_list = []
    g_website_list = []
    g_original_name = []
    g_num_qcc = []
    no_result_list = []
    
#     key_word = input('请输入您想搜索的关键词：')
#     num = int(input('请输入您想检索的次数：'))+1
#     sleep_time = int(input('请输入每次检索延时的秒数：'))
    
    num = 2
    sleep_time = 2.1  
     
    print('正在搜索，请稍后')
     
    df = pd.read_excel ('data/经销商+交易对手方删了最初的.xlsx')
    
    company_list = df.Client
    
#     company_list = ['安徽铂尔商贸有限公司', '安徽铂尔商贸爱上有限公司','安徽铂尔商贸有限公司', '安徽铂尔商贸爱上有限公司']
    
    for i in company_list:

        for x in range(1,num):
            # url = r'https://www.qichacha.com/search_index?key={}&ajaxflag=1&p={}&'.format(key_word,x)
                            
            url = r'https://www.qcc.com/search_index?key={}&ajaxflag=1&p={}&'.format(urllib.parse.quote(i),x)

            s1 = craw(url,i,x)
            
#             g_original_name.append(i)
            
            time.sleep(sleep_time)

        
    workbook = xlwt.Workbook()
    #创建sheet对象，新建sheet
    sheet1 = workbook.add_sheet('企查查数据', cell_overwrite_ok=True)
    #---设置excel样式---
    #初始化样式
    style = xlwt.XFStyle()
    #创建字体样式
    font = xlwt.Font()
    font.name = '仿宋'
#    font.bold = True #加粗
    #设置字体
    style.font = font
    #使用样式写入数据
    print('正在存储数据，请勿打开excel')
    #向sheet中写入数据
    name_list = ['公司名字', '原本表格里面的文字', '企查查搜索结果数', '公司标签','法定法人','注册资本','成立日期','法人邮箱','法人电话','公司地址','公司状态','股东', '网站']

    
    for cc in range(0,len(name_list)):
        sheet1.write(0,cc,name_list[cc],style)
    for i in range(0,len(g_name_list)):
        print(g_name_list[i])
        sheet1.write(i+1,0,g_name_list[i],style)#公司名字
#         sheet1.write(i+1,1,g_original_name[i],style)#原本表格里面的文字
        sheet1.write(i+1,2,g_num_qcc[i],style)#企查查搜索结果数
        sheet1.write(i+1,3,g_tag_list[i],style)#公司标签
        sheet1.write(i+1,4,r_name_list[i],style)#法定法人
        sheet1.write(i+1,5,g_money_list[i],style)#注册资本
        sheet1.write(i+1,6,g_date_list[i],style)#成立日期
        sheet1.write(i+1,7,r_email_list[i],style)#法人邮箱
        sheet1.write(i+1,8,r_phone_list[i],style)#法人电话
        sheet1.write(i+1,9,g_addr_list[i],style)#公司地址
        sheet1.write(i+1,10,g_state_list[i],style)#公司状态
        sheet1.write(i+1,11,g_gudong_list[i],style)#股东信息        
        sheet1.write(i+1,12,g_website_list[i],style)#网站

        
    #保存excel文件，有同名的直接覆盖
    workbook.save(r"desktop"+time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) +".xls")
    # workbook.save(r"D:\wyy-qcc-"+time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) +".xls")
    print('保存完毕~')
    print("没有搜索结果的是", no_result_list)
    
    
# 天水金河源商贸有限公司


正在搜索，请稍后
NO. 1 key_word 七台河利丰源商贸有限公司
re https://www.qcc.com/search?key=%E4%B8%83%E5%8F%B0%E6%B2%B3%E5%88%A9%E4%B8%B0%E6%BA%90%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/64EC397.shtml
GUDONG:  ['徐庆利; ', '赵施梅; ']
NO. 2 key_word 三亚凤凰惠隆冷饮商行
re https://www.qcc.com/search?key=%E4%B8%89%E4%BA%9A%E5%87%A4%E5%87%B0%E6%83%A0%E9%9A%86%E5%86%B7%E9%A5%AE%E5%95%86%E8%A1%8C
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/87TY0Z2.shtml
GUDONG:  ['陈南光; ']
NO. 3 key_word 三明合林贸易有限公司
re https://www.qcc.com/search?key=%E4%B8%89%E6%98%8E%E5%90%88%E6%9E%97%E8%B4%B8%E6%98%93%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/830GLN7.shtml
GUDONG:  ['蔡琦; ', '吴合林; ']
NO. 4 key_word 三河市俊鑫奕城商贸有限公司
re https://www.qcc.com/search?key=%E4%B8%89%E6%B2%B3%E5%B8%82%E4%BF%8A%E9%91%AB%E5%A5%95%E5%9F%8E%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/6

NO. 29 key_word 东阳市海纳食品有限公司
re https://www.qcc.com/search?key=%E4%B8%9C%E9%98%B3%E5%B8%82%E6%B5%B7%E7%BA%B3%E9%A3%9F%E5%93%81%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/57I792C.shtml
GUDONG:  ['蒋益强; ', '王丽妹; ']
NO. 30 key_word 东阿县昊鑫商贸有限公司
re https://www.qcc.com/search?key=%E4%B8%9C%E9%98%BF%E5%8E%BF%E6%98%8A%E9%91%AB%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/30LBTXI.shtml
GUDONG:  ['郝峰; ', '吴文月; ']
NO. 31 key_word 丽水福顺商贸有限公司
re https://www.qcc.com/search?key=%E4%B8%BD%E6%B0%B4%E7%A6%8F%E9%A1%BA%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
好像被拒绝访问了呢...请稍后再试叭...
NO. 32 key_word 丽江市古城区百优商贸有限公司
re https://www.qcc.com/search?key=%E4%B8%BD%E6%B1%9F%E5%B8%82%E5%8F%A4%E5%9F%8E%E5%8C%BA%E7%99%BE%E4%BC%98%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/63BMEEY.shtml
GUDONG:  ['刘志松; ', '蒋红红; ']
NO. 33 key_word 中山市华

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/96NCHEK.shtml
GUDONG:  ['杨继洲; ', '王方; ', '张锟; ']
NO. 61 key_word 信阳凯旋实业有限公司
re https://www.qcc.com/search?key=%E4%BF%A1%E9%98%B3%E5%87%AF%E6%97%8B%E5%AE%9E%E4%B8%9A%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/31U82E3.shtml
GUDONG:  ['章林云; ', '李文杰; ']
NO. 62 key_word 信阳市萱盛实业有限公司
re https://www.qcc.com/search?key=%E4%BF%A1%E9%98%B3%E5%B8%82%E8%90%B1%E7%9B%9B%E5%AE%9E%E4%B8%9A%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/61CUMIY.shtml
GUDONG:  ['于立红; ', '朱学富; ', '李文杰; ']
NO. 63 key_word 兰州乾圣商贸有限公司
re https://www.qcc.com/search?key=%E5%85%B0%E5%B7%9E%E4%B9%BE%E5%9C%A3%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/87NUOSA.shtml
GUDONG:  ['罗艳琴; ', '李宝国; ']
NO. 64 key_word 兰州动达商贸有限公司
re https://www.qcc.com/search?key=%E5%85%B0%E5%B7%9E%E5%8A%A8%E8%BE%BE%E5%95%86%E8%B4%B8%E6%9C%89%E9%99

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/470YPAY.shtml
GUDONG:  ['张毅; ', '朱宝利; ']
NO. 91 key_word 北京永创安信商贸有限公司
re https://www.qcc.com/search?key=%E5%8C%97%E4%BA%AC%E6%B0%B8%E5%88%9B%E5%AE%89%E4%BF%A1%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/44OKQDG.shtml
GUDONG:  ['张娜; ', '郭子谊; ']
NO. 92 key_word 北京沃茂盛商贸有限公司
re https://www.qcc.com/search?key=%E5%8C%97%E4%BA%AC%E6%B2%83%E8%8C%82%E7%9B%9B%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/79HHGCF.shtml
GUDONG:  ['石少雄; ', '孔令彬; ']
NO. 93 key_word 北京温馨鸿达商贸有限公司
re https://www.qcc.com/search?key=%E5%8C%97%E4%BA%AC%E6%B8%A9%E9%A6%A8%E9%B8%BF%E8%BE%BE%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/24F6LTZ.shtml
GUDONG:  ['蒋文彪; ', '王金荣; ']
NO. 94 key_word 北京澳德物流有限责任公司
re https://www.qcc.com/search?key=%E5%8C%97%E4%BA%AC%E6%BE%B3%E5%BE%B7%E7%89%A9

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/21O8PUW.shtml
GUDONG:  ['李星慧; ', '李星杰; ', '李继光; ']
NO. 123 key_word 呼和浩特市精德食品有限公司
re https://www.qcc.com/search?key=%E5%91%BC%E5%92%8C%E6%B5%A9%E7%89%B9%E5%B8%82%E7%B2%BE%E5%BE%B7%E9%A3%9F%E5%93%81%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/65MNEV2.shtml
GUDONG:  ['秦国贞; ', '康健; ', '李亚峰; ', '杨爽利; ']
NO. 124 key_word 咸宁胜思商贸有限公司
re https://www.qcc.com/search?key=%E5%92%B8%E5%AE%81%E8%83%9C%E6%80%9D%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/87PQV3X.shtml
GUDONG:  ['陈敏; ', '周胜; ']
NO. 125 key_word 哈尔滨东霆商贸有限公司
re https://www.qcc.com/search?key=%E5%93%88%E5%B0%94%E6%BB%A8%E4%B8%9C%E9%9C%86%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/1872I76.shtml
GUDONG:  ['姜希阳; ', '董卉; ']
NO. 126 key_word 哈尔滨中天盛合商贸有限公司
re https://www.qcc.com/search?key=%E5%93%88%E5%B0%94%E6%BB%

...

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/47V9RRZ.shtml
GUDONG:  ['张娟; ', '董小兵; ']
NO. 370 key_word 滦南县海兴乳品经销处
re https://www.qcc.com/search?key=%E6%BB%A6%E5%8D%97%E5%8E%BF%E6%B5%B7%E5%85%B4%E4%B9%B3%E5%93%81%E7%BB%8F%E9%94%80%E5%A4%84
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/24CU7V2.shtml
GUDONG:  ['郑淑兰; ']
NO. 371 key_word 滦平富雅赫兴商贸有限公司
re https://www.qcc.com/search?key=%E6%BB%A6%E5%B9%B3%E5%AF%8C%E9%9B%85%E8%B5%AB%E5%85%B4%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/95CEMRH.shtml
GUDONG:  ['孙玉军; ', '罗秀印; ']
NO. 372 key_word 滨州鹏仔商贸有限公司
re https://www.qcc.com/search?key=%E6%BB%A8%E5%B7%9E%E9%B9%8F%E4%BB%94%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
好像被拒绝访问了呢...请稍后再试叭...
NO. 373 key_word 漯河市庞博商贸有限公司
re https://www.qcc.com/search?key=%E6%BC%AF%E6%B2%B3%E5%B8%82%E5%BA%9E%E5%8D%9A%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/582XXRE.shtml
GUDONG:  ['许国旺; ', '尹燕红; ']
NO. 400 key_word 福州爱华乐农贸易有限公司
re https://www.qcc.com/search?key=%E7%A6%8F%E5%B7%9E%E7%88%B1%E5%8D%8E%E4%B9%90%E5%86%9C%E8%B4%B8%E6%98%93%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/662LG4W.shtml
GUDONG:  ['金锐全; ', '薛贤乐; ']
NO. 401 key_word 福建恒正佳华食品有限公司
re https://www.qcc.com/search?key=%E7%A6%8F%E5%BB%BA%E6%81%92%E6%AD%A3%E4%BD%B3%E5%8D%8E%E9%A3%9F%E5%93%81%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/90M2R9P.shtml
GUDONG:  ['陈清华; ', '陈清光; ']
NO. 402 key_word 福清思必得贸易有限公司
re https://www.qcc.com/search?key=%E7%A6%8F%E6%B8%85%E6%80%9D%E5%BF%85%E5%BE%97%E8%B4%B8%E6%98%93%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/45VKXHQ.shtml
GUDONG:  ['薛贤乐; ', '曾秀利; ', '薛华娟; ']
NO. 403 key_word 福清思必得贸易有限公司
re https://www.qcc.com/search?key=%E7%A6%8F%E6%B8%85%E6%80%9D%E5%

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/80TYNA0.shtml
GUDONG:  ['李臣耿; ', '岳元洋; ']
NO. 429 key_word 蓬莱聚秦商贸有限公司
re https://www.qcc.com/search?key=%E8%93%AC%E8%8E%B1%E8%81%9A%E7%A7%A6%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/16KLQLE.shtml
GUDONG:  ['杜福全; ', '刘霞; ']
NO. 430 key_word 衡水市嘉沛商贸有限公司
re https://www.qcc.com/search?key=%E8%A1%A1%E6%B0%B4%E5%B8%82%E5%98%89%E6%B2%9B%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/92UZ5R7.shtml
GUDONG:  ['宁重阳; ', '马晓红; ']
NO. 431 key_word 衡水市天乐百货商贸有限公司
re https://www.qcc.com/search?key=%E8%A1%A1%E6%B0%B4%E5%B8%82%E5%A4%A9%E4%B9%90%E7%99%BE%E8%B4%A7%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/38WEPJJ.shtml
GUDONG:  ['司志成; ', '张彩虹; ']
NO. 432 key_word 衡水广柏商贸有限公司
re https://www.qcc.com/search?key=%E8%A1%A1%E6%B0%B4%E5%B9%BF%E6%9F%8F%E5%95%86%E8%B4%

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/51A3BVF.shtml
GUDONG:  ['杨齐良; ', '王小虎; ']
NO. 460 key_word 郴州市鑫雅商贸有限公司
re https://www.qcc.com/search?key=%E9%83%B4%E5%B7%9E%E5%B8%82%E9%91%AB%E9%9B%85%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/36MQCK3.shtml
GUDONG:  ['欧阳艳琴; ', '肖若琪; ']
NO. 461 key_word 郸城县艳丽食品商贸有限公司
re https://www.qcc.com/search?key=%E9%83%B8%E5%9F%8E%E5%8E%BF%E8%89%B3%E4%B8%BD%E9%A3%9F%E5%93%81%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/216UE3P.shtml
GUDONG:  ['从振涛; ', '崔苏静; ']
NO. 462 key_word 鄂州市好牛商贸有限公司
re https://www.qcc.com/search?key=%E9%84%82%E5%B7%9E%E5%B8%82%E5%A5%BD%E7%89%9B%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/14MB4DJ.shtml
GUDONG:  ['邢小雷; ', '许峥强; ']
NO. 463 key_word 鄄城县兴睿食品有限公司
re https://www.qcc.com/search?key=%E9%84%84%E5%9F%8E%E5%8E%BF%E5%85%B4%E7

开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/523G03U.shtml
GUDONG:  ['王磊; ', '侯胜国; ']
NO. 492 key_word 雅玛多国际货运物流有限公司天津分公司
re https://www.qcc.com/search?key=%E9%9B%85%E7%8E%9B%E5%A4%9A%E5%9B%BD%E9%99%85%E8%B4%A7%E8%BF%90%E7%89%A9%E6%B5%81%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8%E5%A4%A9%E6%B4%A5%E5%88%86%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/841O37B.shtml
GUDONG:  ['松本光市（MATSUMOTO KOICHI）; ']
NO. 493 key_word 青岛万事晟远商贸有限公司
re https://www.qcc.com/search?key=%E9%9D%92%E5%B2%9B%E4%B8%87%E4%BA%8B%E6%99%9F%E8%BF%9C%E5%95%86%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/59CSV34.shtml
GUDONG:  ['仇庆春; ', '高晓丽; ']
NO. 494 key_word 青岛同兴永泰工贸有限公司
re https://www.qcc.com/search?key=%E9%9D%92%E5%B2%9B%E5%90%8C%E5%85%B4%E6%B0%B8%E6%B3%B0%E5%B7%A5%E8%B4%B8%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8
开始爬取数据，请勿打开excel
$$$$$$$$$$$ https://www.qcc.com//firm/10PQ5FT.shtml
GUDONG:  ['袁全友; ', '隋晓君; ']
NO. 495 key_word 青岛同联食品有

In [18]:
# 以一个静态 html 作为例子

html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">There will have more story</p>
"""

# soup = BeautifulSoup(open('天水金河源商贸有限公司-赵卫军_企业工商信息查询-企查查.htm'))
soup = BeautifulSoup(open('阿里巴巴（中国）网络技术有限公司-戴珊_企业工商信息查询-企查查.htm'))

print (soup.prettify())


In [93]:
# 具体看怎么去爬

soup.find_all("h3", class_='seo font-14')[0]

<h3 class="seo font-14">淘宝（中国）软件有限公司</h3>

In [61]:
soup.find_all("h3", class_='seo font-14')[0].text

'淘宝（中国）软件有限公司'

In [18]:
# 爬虫的个例子

#输出第一个 title 标签
print ("soup.title: ", soup.title)
 
#输出第一个 title 标签的标签名称
print ("soup.title.name: ", soup.title.name)
 
#输出第一个 title 标签的包含内容
print ("soup.title.string: ", soup.title.string)
 
#输出第一个 title 标签的父标签的标签名称
print ("soup.title.parent.name: ", soup.title.parent.name)
 
#输出第一个  p 标签
print ("soup.p: ", soup.p)
 
#输出第一个  p 标签的 class 属性内容
print ("soup.p['class']: ", soup.p['class'])
 
#输出第一个  a 标签的  href 属性内容
print ("soup.a['href']: ", soup.a['href'])

'''
soup的属性可以被添加,删除或修改. 再说一次, soup的属性操作方法与字典一样
'''

#修改第一个 a 标签的href属性为 http://www.baidu.com/
soup.a['href'] = 'http://www.baidu.com/'
 
#给第一个 a 标签添加 name 属性
soup.a['name'] = u'百度'
 
#删除第一个 a 标签的 class 属性为
del soup.a['class']
 
##输出第一个  p 标签的所有子节点
print (soup.p.contents)
       
#输出第一个  a 标签
print (soup.a)
 
#输出所有的  a 标签，以列表形式显示
print (soup.find_all('a'))
 
#输出第一个 id 属性等于  link3 的  a 标签
print (soup.find(id="link3"))
 
#获取所有文字内容
print(soup.get_text())
 
#输出第一个  a 标签的所有属性信息
print (soup.a.attrs)
 
for link in soup.find_all('a'):
    #获取 link 的  href 属性内容
    print(link.get('href'))
 
#对soup.p的子节点进行循环输出    
for child in soup.p.children:
    print(child)
 
#正则匹配，名字中带有b的标签
for tag in soup.find_all(re.compile("b")):
    print(tag.name)

    

soup.title:  <title>The Dormouse's story</title>
soup.title.name:  title
soup.title.string:  The Dormouse's story
soup.title.parent.name:  head
soup.p:  <p class="title" name="dromouse"><b>The Dormouse's story</b></p>
soup.p['class']:  ['title']
soup.a['href']:  http://example.com/elsie
[<b>The Dormouse's story</b>]
<a href="http://www.baidu.com/" id="link1" name="百度"><!-- Elsie --></a>
[<a href="http://www.baidu.com/" id="link1" name="百度"><!-- Elsie --></a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
The Dormouse's story

The Dormouse's story
Once upon a time there were three little sisters; and their names were
,
Lacie and
Tillie;
and they lived at the bottom of a well.
There will have more story

{'href': 'http://www.baidu.com/', 'id': 'link1', 'name': '百度'}
http://www.baidu.com/
http://example.com/lacie
http: